In [1]:
import pandas as pd

In [6]:
label = pd.read_csv(r'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\artifacts\transform_roi_data\0\both\0_0.csv', index_col=0)
data = pd.read_csv(r'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\artifacts\raw_data\0\both\0_0.csv')


In [9]:
data['label'] = label['0']

In [13]:
data[data['label'] == 1].to_numpy().shape

(691, 7)